In [75]:
import handcalcs.render
from handcalcs.decorator import handcalc
import forallpeople
forallpeople.environment('structural', top_level=True)

from math import sqrt, sin, cos, tan

# 📖🖊Lesson 07: Engineering Calculation Scripts in Jupyter

Because of the rich media display capabilities of JupyterLab, we can create beautifully formatted calculation scripts for engineering easily.

With a little imagination and cunning, you can do some creative work!

There are three primary tools you can use to _jazz_ up your notebooks for engineering:

1. handcalcs
2. forallpeople
3. IPython display

# Introducing: handcalcs

[Read the handcalcs documentation](https://github.com/connorferster/handcalcs)

**Typical import**
```python
import handcalcs.render
```

After we have done `import handcalcs.render`, we have access to the `%%render` "cell magic". 

A cell magic is a special _Jupyter-only_ command that performs some "magic" ability to the code in your cell before or after running it. 

A normal cell:
```python
a = 4
b = 5.2
c = 2*a + b
```

Lets add the `%%render` cell magic to our previous cell (and ditch the `print()` statement):

```python
%%render
a = 4
b = 5.2
c = 2*a + b
```

Copy-paste the code above into the cell below to see what happens.

# handcalcs: what it's all about

`handcalcs` is about rendering python arithmetic expressions similar to how you would write them out by hand:
1. Display the symbolic formula
2. Followed by the numeric substitutions
3. Followed by the result

While there are other excellent software programs for doing similar stuff (SMath Studio, MathCAD), handcalcs is different in three ways:

1. It shows the numeric substitution
2. You don't have to click and drag your formulas around on the page: it's just laid out nicely
3. It's free and open-source

## How it works

When you run your cell, handcalcs first reads the code in your cell (as thought it were in a text file) and tells Python to run your code.

It splits the numbers and operators apart and then uses several `for` loops to _transform_ each of the python code strings into $\LaTeX$ strings. 

Finally, it gets the calculated results from Python and puts the appropriate result at the end of your calculation code.

JupyterLab comes with `KaTex` installed, which is a web-based Latex renderer developed by Khan Academy.

# handcalcs cell commands

When using `%%render` there are some additional commands you can put afterwards that alters the behaviour of how handcalcs displays your calculation:

* `params` - Does not render the formula or the subsitution. Just displays the variable (symbol) and the value (result) within a three column layout. Useful for displaying input parameters in a condensed fashion or any value for which the calculation is trivial.
* `symbolic` - Does not render the numeric substition or the result. Renders the variable (symbol) and the formula.
* `short` - Some formulas are really long and do not fit on one line. handcalcs tries to guess if your equation is too long and, if so, it will break it up over three lines. Sometimes it guesses wrong. If you want your formula to displayed on one line _as though it were a "short" equation_ then use `short`.
* `long` - Similar to above. If your equation is quite long but handcalcs guesses that it is short, then use `long` to display it over three lines _as though it were a "long" equation_.
* `sympy` - Useful when you are doing algebraic manipulation with sympy expressions. Use `sympy` when you are ready to subsitute numerical values for your sympy symbols.
* `<n>` - Can be used on it's own or with `params`, `short`, and `long`. handcalcs defaults to only showing three decimals of precision. You can enter an integer as a cell command to change the number of decimal places displayed by the numbers _in that cell_.

Try using each of the different cell commands on the following cells to see how they work:

Cell 1:
```python
%%render
a = 3.238728302
b = 4.38
c = -15.0

x_pos = (-b + sqrt(b**2 - 4 * a * c)) / (2*a)
x_neg = (-b - sqrt(b**2 - 4 * a * c)) / (2*a)
Delta = (-b + sqrt(b**2 - 4 * a * c)) / (2*a) - (-b - sqrt(b**2 - 4 * a * c)) / (2*a)
```

Cell 2:
```python
%%render
phi = 0.65
f_prime_c = 35 # in MPa
beta = 0.18
b_w_beam = 300 # in mm
d_v_beam = 520 # in mm
V_c_beam = phi * beta * sqrt(f_prime_c) * b_w_beam * d_v_beam # N, Cl. 11.3.4
```

## A short summary of handcalcs features

* Use `_` in your variable names to create sub-scripts (and sub-sub-scripts, etc.)
* Use Greek letter names for Greek symbols. e.g. `Delta` for a capital delta, `delta` for little delta.
* You can use functions like `sin()` and `cos()` imported from the `math` module
* Python comments are rendered in parentheses after the results. You can use additional $\LaTeX$ code in your comments if surrounded by `$...$`
* You can also use `if`, `elif`, and `else` statements in your cell but you have to write each on one line, e.g.
```python
%%render
a = 2
b = 3
if a > b: c = a * b
elif a < b: c = a + b
```

**A Known Bug (that needs fixing)**

Always put spaces around your equal signs, e.g. `a = 32` instead of `a=32`. Sometimes it gets messed up if you don't.

## Use handcalcs in functions with the `@handcalc` decorator

**Typical import**
```python
from handcalcs.decorator import handcalc
```

While the `%%render` cell magic can only be used in Jupyter and not general Python, you can still use handcalcs in general Python by using the `@handcalc` decorator. It works like this:

```python
@handcalc()
def LC3a(DL: float, LL: float, SL: float) -> float:
    """NBCC Loadcase 3a"""
    FL = 1.25*DL + 1.5*SL + 1.0*LL
    return FL
```

Similar to a cell magic, a decorator is a simply a function that gets called on your function before your function executes.

When the function executes, you will now get two values: 
1. The $\LaTeX$ code, as a `str`
2. The return value from your function

To have the $\LaTeX$ code be automatically rendered in Jupyter, you can add the argument `jupyter_display=True`, as such:

```python
@handcalc(jupyter_display=True)
def LC3a(DL: float, LL: float, SL: float) -> float:
    """NBCC Loadcase 3a"""
    FL = 1.25*DL + 1.5*SL + 1.0*LL
    return FL
```

Now your function will only return one value, your return value, and the _function code_ will be rendered in the cell.

# forallpeople: adding unit awareness

[Read the forallpeople documentation](https://github.com/connorferster/forallpeople/)

**Typical imports**
```python
import forallpeople
forallpeople.environment('structural', top_level=True)
```

Working with `handcalcs` is another library called `forallpeople`. It is so named because the words of Nicholas de Condorcet as he wrote about the new metric system in revolutionary France: "It is to be for all people, for all time."

`forallpeople` is a library that adds capability to perform units-aware calculations by using the SI units system (and units derived or defined by the SI unit system, such as US Customary units).

## What units are there in the `structural` environment?

To find out what units you have access to now, use `forallpeople.environment()` and they will all print to the screen.

## How to use

Simply multiply your values by your units:

```python
phi = 0.65
Z_x = 600e3*mm**3
f_y = 350*MPa
M_r = phi * Z_x * f_y
M_r
```

Use it with handcalcs:

**Cell 1**
```python
%%render params
phi = 0.65
Z_x = 600e3*mm**3
f_y = 40*MPa
```
**Cell 2**
```python
%%render
M_r = phi * Z_x * f_y
```

In [76]:
%%render params
phi = 0.65
Z_x = 600e3*mm**3
f_y = 300*MPa

<IPython.core.display.Latex object>

In [77]:
%%render
M_r = phi * Z_x * f_y

<IPython.core.display.Latex object>

# IPython display functions

Jupyter comes with many ways of displaying rich media. Some of these media types include:

* Markdown
* HTML
* Latex
* Image (PNG, JPG, GIF, etc.)
* SVG (Scaleable Vector Graphics)
* Video
* Audio
* Code

**Typical imports**
```python
from IPython.display import Markdown, HTML, Image, SVG, Video # etc.
from IPython.display import display
```

In [88]:
from IPython.display import Markdown, HTML, Image, SVG, Video # etc.
from IPython.display import display

In [89]:
my_markdown_message = Markdown("**This text is bold**")

In [90]:
display(my_markdown_message)

**This text is bold**